### Read the binary table, HDU #1

In [ ]:
from astropy.io import fits

# this uses the url for the file
# but can alternatively use a local file instead
base_url = 'https://lasp.colorado.edu/eve/data_access/eve_data/products/level0b/megs_a'
megsa_url = f'{base_url}/2010/120/MA__L0B_4_2010120_235905_00_001_01.fit.gz'
hdul = fits.open(megsa_url)

# read the binary table in HDU #1
for name_val_pair in zip(hdul[1].data.names, hdul[1].data[0]):
    print(f"{name_val_pair[0]} = {name_val_pair[1]}")

And peek at the header contents for the binary table HDU #1:

In [ ]:
hdul[1].header

We can also examine the contents of the header for the imaage in HDU #0:

In [ ]:
hdul[0].header

### Read and display the image (HDU #0)

In [ ]:
import os
import matplotlib.pyplot as plt
from skimage import exposure
from astropy.io import fits

with fits.open(megsa_url) as hdul:
    image = hdul[0].data    # the image is the first HDU

# histogram equalization is helpful for viewing the features in the image
image_eq = exposure.equalize_hist(image)            
plt.figure(figsize=(12,6))
plt.imshow(image_eq, cmap='gist_heat', origin='lower')
plt.title(os.path.basename(megsa_url), fontsize=16)
plt.show()

<i>Figure 6: A single histogram equalized MEGS-A image with 10-second integration time. The slit 1 spectrum is dispersed across the top with short wavelengths on the right side. The bright Fe IX line at 17.1 nm is the brightest line in slit 1. Slit 2 also shows 17.1 and all of the longer wavelengths to the left. The SAM pinhole camera is in the lower right. Particle spikes and streaks are scattered across the detector.</i>

### Read multiple files

In [ ]:
import numpy as np
import os
from urllib.error import HTTPError

# this gives us five minutes of MEGS-A L0B files
base_url = 'https://lasp.colorado.edu/eve/data_access/eve_data/products/level0b/megs_a'
files = [f'{base_url}/2010/120/MA__L0B_4_2010120_235{x}{y}5_00_001_01.fit.gz' for x in range(5) for y in range(5)]

image_sum = np.zeros((1024,2048))       # accumulate the running sum of each image

# loop over the files and add each image to the running total
# if the file isn't found on the web, skip it
for this_file in files: 
    try:
        with fits.open(this_file) as hdul:
            image_sum += hdul[0].data    # the image is the first HDU
    except HTTPError:
        print(f"File not found: {os.path.basename(this_file)}")
        continue

# display the histogram equalized sum of images
image_sum_eq = exposure.equalize_hist(image_sum)            
plt.figure(figsize=(12,6))
plt.imshow(image_sum_eq, cmap='gist_heat', origin='lower')
plt.title("Level 0B MEGS-A 5-minute image", fontsize=16)
plt.show()

<i>Figure 7: A 5-minute sum of 10-second integrations with histogram equalization makes it easier to see the SAM image and lines transmitted through slit 2. Larger particle spikes and streaks are easily observed when viewing multiple images.</i>

### Plot uncalibrated spectrum

Use the 5-minute image from the last step to plot an uncalibrated spectrum. As noted in the IDL section, the wavelengths are not uniformly distributed and are slighly curved.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12,8))

axes[0].plot(np.arange(1024)+1024, np.median(image_sum[800:808,1024:], axis=0))
axes[1].plot(np.arange(1200), np.median(image_sum[300:308,:1200], axis=0))

for i, ax in enumerate(axes):
    ax.grid(linestyle='dotted', zorder=0, color='grey')
    ax.set_title(f'Slit {i+1}')
    ax.set_ylabel('Arbitrary')
    ax.autoscale(enable=True, axis='x', tight=True)
axes[1].set_xlabel('Pixel')

plt.show()

<i>Figure 8: Uncalibrated spectra from MEGS-A slit 1 and 2 near the centers of each slit. The vertical axis has arbitrary units, and the horizontal axis is a reversed non-linear function of wavelength. Median filtering was applied in cross-dispersion to reduce the effect of particle strikes.</i>

### Read a MEGS-B file and display the image

In [ ]:
mb_file_url = 'https://lasp.colorado.edu/eve/data_access/eve_data/products/level0b/megs_b/2010/123/MB__L0B_3_2010123_180006_00_001_01.fit.gz'

with fits.open(mb_file_url) as hdul:
    image = hdul[0].data

image_eq = exposure.equalize_hist(image)
plt.figure(figsize=(12,6))
plt.imshow(image_eq, cmap='Blues_r', origin='lower')
plt.title(os.path.basename(mb_file_url), fontsize=16)
plt.show()

<i>Figure 9: MEGS-B 10-second solar spectrum on the first rocket day, 2010 day 123, near 18:00 UTC.</i>

More detailed information about the images and features is in the IDL section of the Level 0B readme.